Aprendizaje Automático - Práctica 1
Autores: RICARDO ANTONIO PAZOS VALERO - 100472303 / VICENTE ANTONIO BARBATO - 10043114

# 6. Problema de clasificación

### Primera Parte: Usar modelo final para comprobar predicciones de valores altos y valores bajos

In [10]:
# Importamos
from joblib import load
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Cargamos datos
data = pd.read_csv('data/wind_ava.csv')

# Cargamos el modelo final
final_model = load('modelo_final.pkl')

# Filtramos los datos para ver solo los datos metereológicos obtenidos para Sotavento 
data_filter = [columna for columna in data.columns if columna.endswith('.13') or columna == "energy"]
data_sotavento = data[data_filter]
data_sotavento = data_sotavento.copy()  # Hacemos una copia de los datos para evitar el SettingWithCopyWarning

# Definimos el tercer cuantil
energy_tq = data_sotavento['energy'].quantile(0.75)

# Creamos una nueva variable target - la definimos como una variable binaria (más fácil de trabajar con variables binarias)
# Clase alta = 1
# Clase baja = 0
data_sotavento['energy_level'] = (data_sotavento['energy'] > energy_tq).astype(int)

# Hacemos las predicciones con el modelo final
predictions = final_model.predict(data_sotavento.drop(columns=['energy', 'energy_level']))

# Separamos los datos en dos subsets de energía alta o energía baja
high_energy = data_sotavento[data_sotavento['energy_level'] == 1]
low_energy = data_sotavento[data_sotavento['energy_level'] == 0]

# Hacemos predicciones para cada subset por separado
high_energy_predictions = predictions[high_energy.index]
low_energy_predictions = predictions[low_energy.index]

# Evaluamos el modelo en cada subset por separado
high_energy_mae = mean_absolute_error(high_energy['energy'], high_energy_predictions)
high_energy_mse = mean_squared_error(high_energy['energy'], high_energy_predictions)
high_energy_r2 = r2_score(high_energy['energy'], high_energy_predictions)

low_energy_mae = mean_absolute_error(low_energy['energy'], low_energy_predictions)
low_energy_mse = mean_squared_error(low_energy['energy'], low_energy_predictions)
low_energy_r2 = r2_score(low_energy['energy'], low_energy_predictions)

# Comparamos métricas
print("Métricas - Energía alta:")
print("MAE:", high_energy_mae)
print("MSE:", high_energy_mse)
print("R^2:", high_energy_r2)

print("\nMétricas - Energía baja:")
print("MAE:", low_energy_mae)
print("MSE:", low_energy_mse)
print("R^2:", low_energy_r2)

Métricas - Energía alta:
MAE: 683.7764569877376
MSE: 632524.6271455635
R^2: -2.956877450284961

Métricas - Energía baja:
MAE: 345.8537427064807
MSE: 185738.59314795886
R^2: -0.8795894043467756


Observamos que tenemos mejores resultados en las predicciones de los valores de energía bajos

### Segunda Parte: Cambiar a un modelo de clasificación
Para el problema de clasificación elegimos el DecisionTreeClassifier. Nos interesa utilizar su hiper-parámetro "weight_class" para poder lidiar con el desbalanceo de las clases

In [11]:
# Importamos
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler

# Para simplificar nuestro set de datos, ahora con la variable de energía "alta" = 1 o "baja" = 0, borramos la variable "energy"
data_sotavento = data_sotavento.drop(columns=['energy'])

# Definimos nuestro data (atributos meteorolócios) y nuestro target (clase de energía)
X, y = data_sotavento.filter(regex=".13"), data_sotavento.energy_level

# Hacemos un split de los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=100472303)

# Hacemos nuestro pipeline
clf = Pipeline([
    ('scaler', StandardScaler()),
    ('Decision Tree', DecisionTreeClassifier(class_weight='balanced', random_state=100472303))
])

# Fit
clf.fit(X_train, y_train)

# Predict
y_pred = clf.predict(X_test)

# Calculamos métricas
accuracy = accuracy_score(y_test, y_pred)
precision_recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

# Imprimimos resultados
print(f"Accuracy: {accuracy}\nPrecision-Recall: {precision_recall}\nF1: {f1}\nROC AUC: {roc_auc}\n")

Accuracy: 0.8366305041480536
Precision-Recall: 0.6744186046511628
F1: 0.6709511568123393
ROC AUC: 0.7821245565628694

